In [1]:
#!pip uninstall typing-extensions -y
#!pip install typing-extensions==4.5.0
!pip install spacy -q
!pip install pandas -q

In [2]:
import json
import spacy
import pandas as pd
from spacy.tokens import DocBin
from tqdm import tqdm

In [3]:
column_names = ['Grant Number', 'Content']

# Initialize an empty list to store the lines from the file
lines = []

# Read the file line by line and append each line to the list
with open('sectionii.csv', 'r', encoding='latin-1') as file:
    # Read the header line
    header = file.readline().strip().split(',')
    
    # Process each subsequent line
    for line in file:
        if line.strip():  # Only process non-empty lines
            # Split the line into two parts using a comma as the delimiter
            parts = line.strip().split(',', 1)  # Limit split to 1 to avoid splitting the section2 text
            lines.append(parts)

# Create a DataFrame from the list of lines with column names
df = pd.DataFrame(lines, columns=column_names)

# Print only the head of the DataFrame
print(df.head())

    Grant Number                                            Content
0  RFA-RM-10-008  """II. Award Information 1. Mechanism of Suppo...
1  RFA-MH-14-215  """II. Award Information Funding Instrument Co...
2  RFA-ES-09-004  """II. Award Information 1. Mechanism of Suppo...
3  RFA-AA-12-002  """Section II. Award Information Funding Instr...
4  RFA-AG-14-005  """Section II. Award Information Funding Instr...


In [4]:
nlp = spacy.load("model-best")

In [5]:
def extract_cash_and_years(text):
    # Process the text using the SpaCy pipeline
    doc = nlp(text)
    
    # Initialize variables to store cash and years
    cash = None
    years = None
    
    # Extract entities and assign values to cash and years
    for ent in doc.ents:
        if ent.label_ == 'MONEY':
            cash = ent.text
        elif ent.label_ == 'YEARS':
            years = ent.text
            
    return cash, years

In [6]:
df['cash'], df['years'] = zip(*df['Content'].apply(extract_cash_and_years))

In [7]:
print(df.head())

    Grant Number                                            Content  \
0  RFA-RM-10-008  """II. Award Information 1. Mechanism of Suppo...   
1  RFA-MH-14-215  """II. Award Information Funding Instrument Co...   
2  RFA-ES-09-004  """II. Award Information 1. Mechanism of Suppo...   
3  RFA-AA-12-002  """Section II. Award Information Funding Instr...   
4  RFA-AG-14-005  """Section II. Award Information Funding Instr...   

          cash years  
0      500,000  five  
1         None     3  
2      700,000     5  
3  1.8 million  five  
4      120,000     5  


In [19]:
# Function to convert cash values into digits
def convert_cash_to_digits(cash_value):
    if cash_value is not None:
        cash_value = str(cash_value)
        # Case 1: If it only contains digits and a comma, then remove the comma
        
        if cash_value.replace(',', '').isdigit():
            return int(cash_value.replace(',', ''))
        if cash_value.replace('.', '').isdigit():
            cash_value_no_commas = cash_value.replace(',', '')
            # Convert the cleaned string to a float
            cash_value_float = float(cash_value_no_commas.rstrip("."))
            # Multiply the float value by 1,000,000
            cash_value_in_millions = cash_value_float * 1_000_000
            # Convert the result to an integer
            cash_value_int = int(cash_value_in_millions)
            return cash_value_int
        # Case 2: If it contains letters, check if they are 'million' or 'm'
        elif any(char.isalpha() for char in cash_value):
            cash_value_lower = cash_value.lower()
            if 'million' in cash_value_lower or 'm' in cash_value_lower:
                # Take the digits but keep the point separator (for example 2.4), convert it to float
                # and multiply by 1000000
                cash_value_no_commas = cash_value.replace(',', '')
                # Filter out only digits and the decimal point from the string
                filtered_chars = ''.join(filter(lambda x: x.isdigit() or x == '.', cash_value_no_commas))
                # Convert the filtered string to a float
                cash_value_float = float(filtered_chars.rstrip("."))
                # Multiply the float value by 1,000,000
                cash_value_in_millions = cash_value_float * 1000000
                # Convert the result to an integer
                cash_value_int = int(cash_value_in_millions)
                return cash_value_int
    return None


In [20]:
def test_conversion():
    assert convert_cash_to_digits('250,000') == 250000
    assert convert_cash_to_digits('1,000,000') == 1_000_000
    assert convert_cash_to_digits('1 million') == 1_000_000
    assert convert_cash_to_digits('2.4') == 2_400_000
    assert convert_cash_to_digits('1.0.') == 1_000_000
    

    print("yippie test ssucceeded")
test_conversion()

yippie test ssucceeded


In [21]:
df['cash_formatted'] = df['cash'].apply(convert_cash_to_digits)

In [22]:
print(df)

       Grant Number                                            Content  \
0     RFA-RM-10-008  """II. Award Information 1. Mechanism of Suppo...   
1     RFA-MH-14-215  """II. Award Information Funding Instrument Co...   
2     RFA-ES-09-004  """II. Award Information 1. Mechanism of Suppo...   
3     RFA-AA-12-002  """Section II. Award Information Funding Instr...   
4     RFA-AG-14-005  """Section II. Award Information Funding Instr...   
...             ...                                                ...   
4874  RFA-HG-14-006  """Section II. Award Information Funding Instr...   
4875  RFA-EB-15-002  """Section II. Award Information Funding Instr...   
4876  RFA-DK-14-507  """Section II. Award Information Funding Instr...   
4877  RFA-DK-14-505  """Section II. Award Information Funding Instr...   
4878  RFA-RM-14-017  """Section II. Award Information Funding Instr...   

             cash years  cash_formatted  
0         500,000  five        500000.0  
1            None     3    

In [23]:
df.to_csv('output.csv', index=False)